In [1]:
from helpers import *

In [2]:
base_path = '/Users/rodrse/Downloads/ForSergio_nov26_2023_final'

In [3]:

taz2edge_xml = 'taz_new.xml'
net_xml = 'SFO.net.xml'
fixed_routes_xml = f'{base_path}/5hr_route_choice_set.csv'
od_duration_seconds = 5*60 

# duration of sample time for simulation output statistics
simulation_stat_freq_sec = od_duration_seconds
sim_end_time = od_duration_seconds
additional_xml = f'additional.add_statfreq{od_duration_seconds}.xml'

# suffix of simulation output edge file
EDGE_OUT_STR = 'edge_data_SFO.xml'
TRIPS2ODS_OUT_STR = 'trips.xml'
SUMO_PATH = '/usr/local/opt/sumo/share/sumo'

In [4]:
gt_version_str = 'v4'


# # gt v5:
# mean_od_val = 100
# num_ods = 100

# gt v4:
mean_od_val = 100
num_ods = 10

# # gt v3:
# mean_od_val = 0.05

# # gt v2:
# mean_od_val = 0.5
#num_ods = routes_df.shape[0]


# # gt v1:
# mean_od_val = 100
#num_ods = routes_df.shape[0]


print('if you want to optimize them all (~86k) set num_ods as defined in commented line below')
#num_ods = routes_df.shape[0]

if you want to optimize them all (~86k) set num_ods as defined in commented line below


In [5]:
od_xml = f'gt_od_{gt_version_str}.xml'
file_gt = f'{base_path}/gt_od_{gt_version_str}.xml'
file_gt_edges = f'{base_path}/gt_edges_{gt_version_str}.csv'
prefix_output = f'gt_{gt_version_str}'

In [6]:
# Get GT OD
print("Reading:",file_gt)
tree = ET.parse(file_gt)
root = tree.getroot()
gt_od_df =  xml2df_str(root, 'tazRelation')

gt_od_df.head()

Reading: /Users/rodrse/Downloads/ForSergio_nov26_2023_final/gt_od_v4.xml


,from,to,count,data
0,10100,10400,97.4,None
1,10100,10500,114.9,None
2,10100,105000,104.1,None
3,10100,108100,100.0,None
4,10100,108200,109.1,None


In [7]:
print("Reading:",fixed_routes_xml)
routes_df = pd.read_csv(fixed_routes_xml, index_col=0)

Reading: /Users/rodrse/Downloads/ForSergio_nov26_2023_final/5hr_route_choice_set.csv


In [8]:
od_xml

'gt_od_v4.xml'

In [9]:
prefix_output

'gt_v4'

In [12]:
# simulate gt od
simulate_od(od_xml, 
            prefix_output, 
            base_path, 
            net_xml, 
            taz2edge_xml, 
            additional_xml,
            routes_df,
            sim_end_time,
            TRIPS2ODS_OUT_STR)

od2trips  --spread.uniform --taz-files /Users/rodrse/Downloads/ForSergio_nov26_2023_final/taz_new.xml --tazrelation-files /Users/rodrse/Downloads/ForSergio_nov26_2023_final/gt_od_v4.xml -o /Users/rodrse/Downloads/ForSergio_nov26_2023_final/gt_v4_trips_beforeRteUpdates.xml
Success.time 298.30
###### Running SUMO #######
Seed 0
sumo --output-prefix gt_v4_ --ignore-route-errors=true --net-file=/Users/rodrse/Downloads/ForSergio_nov26_2023_final/SFO.net.xml --routes=/Users/rodrse/Downloads/ForSergio_nov26_2023_final/gt_v4_trips.xml -b 0 -e 300 --additional-files /Users/rodrse/Downloads/ForSergio_nov26_2023_final/additional.add_statfreq300.xml --duration-log.statistics --xml-validation never --vehroutes /Users/rodrse/Downloads/ForSergio_nov26_2023_final/routes.vehroutes.xml --verbose --no-warnings --mesosim true --seed 0
Loading net-file from '/Users/rodrse/Downloads/ForSergio_nov26_2023_final/SFO.net.xml' ... 
done (26234ms).
Loading additional-files from '/Users/rodrse/Downloads/ForSergio_

In [10]:
def od_xml_to_df(file_path):

    tree = ET.parse(file_path)
    root = tree.getroot()
    gt_od_df =  xml2df_str(root, 'tazRelation')
    
    gt_od_vals = gt_od_df['count'].astype(float)
    print('total GT demand: ',gt_od_vals.sum())

    return gt_od_df

In [11]:
gt_od_df = od_xml_to_df(file_gt)

total GT demand:  1044.3999999999999


## Vary OD values adding Gaussian noise

In [34]:
gt_od_df['count'].astype(float).to_numpy()

array([ 97.4, 114.9, 104.1, 100. , 109.1, 106.7,  87.8, 101.1, 113.9,
       109.4])

In [36]:
num_epsilon_iter = 10
ods_epsilon = []

# Base OD which we will update their count entries
base_od = gt_od_df.copy()
gt_od_vals = gt_od_df['count'].astype(float).to_numpy()

for o1 in range(num_epsilon_iter):
    print(f"########### Epsilon {o1} ###########")

    file_od_epsilon_xml = f'gt_od_{gt_version_str}_epsilon{o1}.xml'
    prefix_output = f'epsilon{o1}'

    # Generate OD
    curr_od = gt_od_vals.copy()
    curr_od = np.random.normal(gt_od_vals, gt_od_vals/30)

    print(f'total expected GT demand: {np.sum(curr_od)}')

    ###
    # create OD xml file 
    ###
    base_od['count'] = curr_od
    # round to 1 decimal point
    base_od['count'] = [round(elem, 1) for elem in base_od['count']]     
    base_od = base_od.rename(columns={'fromTaz':'from', 'toTaz':'to'})        
    create_taz_xml(file_od_epsilon_xml, base_od, od_duration_seconds, base_path)
    ods_epsilon.append(curr_od)
    
    # simulate gt od
    simulate_od(file_od_epsilon_xml, 
                prefix_output, 
                base_path, 
                net_xml, 
                taz2edge_xml, 
                additional_xml, 
                routes_df,
                sim_end_time,
                TRIPS2ODS_OUT_STR)

########### Epsilon 0 ###########
total expected GT demand: 1036.1416403039823
Created  /Users/rodrse/Downloads/ForSergio_nov26_2023_final/gt_od_v4_epsilon0.xml
od2trips  --spread.uniform --taz-files /Users/rodrse/Downloads/ForSergio_nov26_2023_final/taz_new.xml --tazrelation-files /Users/rodrse/Downloads/ForSergio_nov26_2023_final/gt_od_v4_epsilon0.xml -o /Users/rodrse/Downloads/ForSergio_nov26_2023_final/epsilon0_trips_beforeRteUpdates.xml
Success.time 298.28
###### Running SUMO #######
Seed 0
sumo --output-prefix epsilon0_ --ignore-route-errors=true --net-file=/Users/rodrse/Downloads/ForSergio_nov26_2023_final/SFO.net.xml --routes=/Users/rodrse/Downloads/ForSergio_nov26_2023_final/epsilon0_trips.xml -b 0 -e 300 --additional-files /Users/rodrse/Downloads/ForSergio_nov26_2023_final/additional.add_statfreq300.xml --duration-log.statistics --xml-validation never --vehroutes /Users/rodrse/Downloads/ForSergio_nov26_2023_final/routes.vehroutes.xml --verbose --no-warnings --mesosim true --s

In [15]:
num_gt_edges = 10
sim_edge_out = f'{base_path}/{prefix_output}_{EDGE_OUT_STR}'
df_edge_gt, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out, prefix_output,SUMO_PATH)

df_edge_gt.head()

,edge_id,interval_nVehContrib,interval_harmonicMeanSpeed
0,-148846998#1,54,6.85
1,-224384010#1,54,13.90
2,-24335486#1,34,11.18
3,-254299125#2,88,13.80
4,-254299128#1,80,11.18


In [17]:
gt_edge_data = df_edge_gt\
        .sort_values(by=['interval_nVehContrib'], ascending=False)\
        .iloc[:num_gt_edges]

In [13]:
edge_epsilon_all = []
num_epsilon_iter = 10
for o1 in range(num_epsilon_iter):
    curr_prefix = f'epsilon{o1}'
    sim_edge_out = f'{base_path}/{curr_prefix}_{EDGE_OUT_STR}'
    curr_loop_stats, _, _ = parse_loop_data_xml_to_pandas(base_path, sim_edge_out, curr_prefix,SUMO_PATH)
    edge_epsilon_all.append(curr_loop_stats)

In [18]:
loss_all = []
for o1 in range(num_epsilon_iter):
    curr_loss = compute_nrmse_counts_all_edges(gt_edge_data, edge_epsilon_all[o1])
    print(curr_loss)
    loss_all.append(curr_loss)


number of GT edges: 10
0.004477033025250183
number of GT edges: 10
0.011462914394989258
number of GT edges: 10
0.005969377367000244
number of GT edges: 10
0.012123862745318669
number of GT edges: 10
0.018876828692779613
number of GT edges: 10
0.008701788067289181
number of GT edges: 10
0.009899092479189726
number of GT edges: 10
0.012750595645215947
number of GT edges: 10
0.009899092479189726
number of GT edges: 10
0.012396343120871824


In [17]:
print("Number of ODs:",num_ods)
gt_od_df = routes_df[['fromTaz', 'toTaz']].drop_duplicates().iloc[:num_ods]

Number of ODs: 10


,fromTaz,toTaz
0,10100,10400
1,10100,10500
2,10100,105000
3,10100,108100
4,10100,108200
5,10100,11200
6,10100,11300
7,10100,114200
8,10100,11700
9,10100,11902
